In [193]:
import numpy as np
import os
import sys
import math
import matplotlib.pyplot as plt
sys.path.append(os.getcwd()+'/../')
sys.path.append(os.getcwd()+'/../nonlinear_solvers')
sys.path.append(os.getcwd()+'/../stat_tests')
from lasso import lasso



rootdir=os.getcwd()

DatapointEnergies=np.loadtxt(open(rootdir+'/SR_LSDAPW92VV10_w0p3-cx0p15-4258_QZVPPD_250974_Energies.csv'))
DatapointJacobian=np.loadtxt(open(rootdir+'/SR_LSDAPW92VV10_w0p3-cx0p15-4258_QZVPPD_250974_Jacobian.csv'),delimiter=",")
RefValues=np.loadtxt(open(rootdir+'/Reference_New.csv'))
WTrainDiagonal=np.loadtxt(open(rootdir+'/WTrainDiagonal_Attempt26.csv'))
WTotalDiagonal=np.loadtxt(open(rootdir+'/WTotalDiagonal_Attempt26.csv'))

In [194]:
RefValues.shape

(3835,)

In [195]:
y = np.array(RefValues - DatapointEnergies)
print y.shape

(3835,)


In [196]:
W = np.array(WTrainDiagonal)
W_test = np.array(WTotalDiagonal - WTrainDiagonal)

In [199]:
X = np.array(DatapointJacobian)
#X = np.column_stack((X[:,1:45],X[:,46:90],X[:,91:135] ))
#X = X[:,[1,2,9,10,11,18,19,20,46,47,48,49,50,51,54,55,56,57,58,59,60,63,64,65,66,67,68,69,72,73,74,75,76,77,78,81,82,83,84,85,86,87,91,92,93,94,95,96,99,100,101,102,103,104,105,108,109,110,111,112,113,114,117,118,119,120,121,122,123,126,127,128,129,130,131,132]]
X = X[:,[9,18,27,36,54,63,72,81,99,108,117,126]]
X.shape

(3835, 12)

In [200]:
metric = np.dot(np.transpose(X),np.dot(np.diag(W),X))
U,s,V = np.linalg.svd(metric) #NB svals are decreasing
print 'the eigenvalues of the metric'
print s
print(np.finfo(float).eps)
tol = 1.0E-8
n_indep = s.size
for i in s:
    if (i < tol):
        n_indep = n_indep - 1
print 'number of nonzero ',n_indep
if n_indep > 0:
    metric_inv = np.dot(U[:,:n_indep],np.dot(np.diag(s[:n_indep]**-1),np.transpose(U[:,:n_indep])))
else:
    print 'poorly posed problem in linear_regression'
    sys.exit(1)
beta = np.dot(metric_inv,np.dot(np.transpose(X),np.dot(np.diag(W),y)))
#beta = np.dot(np.linalg.inv(metric),np.dot(np.transpose(X),np.dot(np.diag(W),y)))
print beta.shape
print beta

the eigenvalues of the metric
[  3.27341527e+01   1.89986063e+00   6.66322145e-01   1.40432005e-01
   2.15573554e-02   1.28957633e-02   7.04692727e-04   2.61389830e-04
   1.05767165e-04   2.82932412e-06   1.47026285e-06   6.83223656e-07]
2.22044604925e-16
number of nonzero  12
(12,)
[  0.467761     4.94580422 -16.03957887  19.79831962  -9.84465708
  37.08622631 -51.3879185   23.29833142   3.45473546 -25.02185549
  42.8550738  -25.82588536]


In [182]:
coef_v_lambda = []
test_v_lambda = []
train_v_lambda = []

In [189]:
lambdas = np.linspace(0.000,0.02,21,endpoint=True)
print lambdas

[ 0.     0.001  0.002  0.003  0.004  0.005  0.006  0.007  0.008  0.009
  0.01   0.011  0.012  0.013  0.014  0.015  0.016  0.017  0.018  0.019
  0.02 ]


In [190]:
lasso_obj = lasso(X,y,False,W,True,True)

In [191]:
lasso_obj.obtain_guess_for_lambda(lambdas[0])

In [192]:
for l in lambdas:
      converged = lasso_obj.train(l,False)
      if converged:
         y_pred = np.dot(X,lasso_obj.beta)
         err = y-y_pred
         train_rms = math.sqrt(np.dot(err,np.dot(np.diag(W),err))/float(1072))
         test_rms = math.sqrt(np.dot(err,np.dot(np.diag(W_test),err))/float(len(W)-1072))
         coef_v_lambda.append(lasso_obj.beta)
         test_v_lambda.append(test_rms)
         train_v_lambda.append(train_rms)
         print 'lambda = '+str(l)+' train err = '+str(train_rms)+' test err = '+str(test_rms)
         print lasso_obj.beta
      else:
         print 'failed to converge for lambda ='+str(l)
         coef_v_lambda.append(np.zeros(1))
         test_v_lambda.append(-1.0)
         train_v_lambda.append(-1.0)


beginning minimization of LASSO objective function with lambda = 0.0
  1  0.042661431247  1.463582211e-09  Optimization Converged
lambda = 0.0 train err = 0.0063084159056 test err = 0.00811926129676
[  0.467761     4.94580422 -16.03957887  19.79831962  -9.84465708
  37.08622631 -51.3879185   23.29833142   3.45473546 -25.02185549
  42.8550738  -25.82588536]
beginning minimization of LASSO objective function with lambda = 0.001
  1  0.302687578387  0.000999999576151  Line Search Step
  2  0.302711951896  0.0193837851158  Line Search Step
  3  0.302690671766  0.00957348712999  Line Search Step
  4  0.302686851733  0.00470609720217  BFGS Step
  5  0.302684821454  0.00300542798034  BFGS Step
  6  0.302676187039  0.00249467749141  BFGS Step
  7  0.302664736337  0.00621421824439  BFGS Step
  8  0.302647439101  0.00802563639043  BFGS Step
  9  0.302596696245  0.0123287520014  BFGS Step
  10  0.302410168721  0.0208512622289  BFGS Step
  11  0.302207724354  0.0214160588194  BFGS Step
  12  0.302

KeyboardInterrupt: 